In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
# import matplotlib.pyplot as plt
from math import *
import time

In [2]:
torch.cuda.set_device(2)
torch.set_default_tensor_type('torch.DoubleTensor')

In [3]:
class Net(torch.nn.Module):
    def __init__(self,input_width,layer_width):
        super(Net,self).__init__()
        self.layer_out = torch.nn.Linear(layer_width, 1)
    def forward(self,x):
        output = self.layer_out(x)
        return output

In [4]:
dimension = 1

In [5]:
input_width,layer_width = dimension, 1

In [6]:
net = Net(input_width,layer_width).cuda() # network for u on gpu

In [7]:
# defination of exact solution
def u_ex(x):     
    u_temp = 2*x**3 - x**2 - x
    return u_temp.reshape([x.size()[0], 1])

In [8]:
# defination of f(x)
def f(x):
    f_temp = -(12*x - 2)
    return f_temp.reshape([x.size()[0],1])

In [9]:
# generate points by random
def generate_sample(data_size):
    sample_temp = torch.rand(data_size, dimension)
    return sample_temp.cuda()

In [10]:
def model(x):
    x_temp = x.cuda()
    D_x_0 = torch.prod(x_temp, axis = 1).reshape([x.size()[0], 1]) 
    D_x_1 = torch.prod(1.0 - x_temp, axis = 1).reshape([x.size()[0], 1]) 
    model_u_temp = D_x_0 * D_x_1 * net(x)
    return model_u_temp.reshape([x.size()[0], 1])

In [11]:
# Xavier normal initialization for weights:
#             mean = 0 std = gain * sqrt(2 / fan_in + fan_out)
# zero initialization for biases
def initialize_weights(self):
    for m in self.modules():
        if isinstance(m,nn.Linear):
            nn.init.xavier_normal_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()

In [12]:
initialize_weights(net)

In [13]:
# loss function to DRM by auto differential
def loss_function(x):
#     x = generate_sample(data_size).cuda()
#     x.requires_grad = True
    u_hat = model(x)
    grad_u_hat = torch.autograd.grad(outputs = u_hat, inputs = x, grad_outputs = torch.ones(u_hat.shape).cuda(), create_graph = True)
    grad_u_sq = ((grad_u_hat[0]**2).sum(1)).reshape([len(grad_u_hat[0]), 1])
    part = torch.sum(0.5 * grad_u_sq  - f(x) * u_hat)  / len(x)
    return part

In [14]:
def relative_l2_error():
    data_size_temp = 500
    x = generate_sample(data_size_temp).cuda() 
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [15]:
optimizer = optim.Adam(net.parameters())

In [16]:
epoch = 10000
data_size = 10000
loss_record = np.zeros(epoch)
error_record = np.zeros(epoch)
time_start = time.time()
for i in range(epoch):
    optimizer.zero_grad()
    x = generate_sample(data_size).cuda()
    x.requires_grad = True
    loss = loss_function(x)
    loss_record[i] = float(loss)
    error = relative_l2_error()
    error_record[i] = float(error)
    np.save("unit_DGM_loss_3d.npy", loss_record)
    np.save("unit_DGM_error_3d.npy", error_record)
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", loss.detach())
        print("current error is: ", error.detach())
    if i == epoch - 1:
        # save model parameters
        torch.save(net.state_dict(), 'net_params_DRM.pkl')
        
    loss.backward()
    optimizer.step() 
    torch.cuda.empty_cache() # clear memory
    
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(-0.2066, device='cuda:2')
current error is:  tensor(0.8644, device='cuda:2')
current epoch is:  50
current loss is:  tensor(-0.2570, device='cuda:2')
current error is:  tensor(0.8277, device='cuda:2')
current epoch is:  100
current loss is:  tensor(-0.2950, device='cuda:2')
current error is:  tensor(0.7898, device='cuda:2')
current epoch is:  150
current loss is:  tensor(-0.3323, device='cuda:2')
current error is:  tensor(0.7575, device='cuda:2')
current epoch is:  200
current loss is:  tensor(-0.3800, device='cuda:2')
current error is:  tensor(0.7215, device='cuda:2')
current epoch is:  250
current loss is:  tensor(-0.4006, device='cuda:2')
current error is:  tensor(0.6885, device='cuda:2')
current epoch is:  300
current loss is:  tensor(-0.4496, device='cuda:2')
current error is:  tensor(0.6554, device='cuda:2')
current epoch is:  350
current loss is:  tensor(-0.4757, device='cuda:2')
current error is:  tensor(0.6231, device='cuda:2')
cur

current epoch is:  3300
current loss is:  tensor(-0.7605, device='cuda:2')
current error is:  tensor(0.0156, device='cuda:2')
current epoch is:  3350
current loss is:  tensor(-0.7698, device='cuda:2')
current error is:  tensor(0.0152, device='cuda:2')
current epoch is:  3400
current loss is:  tensor(-0.7443, device='cuda:2')
current error is:  tensor(0.0149, device='cuda:2')
current epoch is:  3450
current loss is:  tensor(-0.7458, device='cuda:2')
current error is:  tensor(0.0145, device='cuda:2')
current epoch is:  3500
current loss is:  tensor(-0.7706, device='cuda:2')
current error is:  tensor(0.0142, device='cuda:2')
current epoch is:  3550
current loss is:  tensor(-0.7891, device='cuda:2')
current error is:  tensor(0.0135, device='cuda:2')
current epoch is:  3600
current loss is:  tensor(-0.7438, device='cuda:2')
current error is:  tensor(0.0129, device='cuda:2')
current epoch is:  3650
current loss is:  tensor(-0.7706, device='cuda:2')
current error is:  tensor(0.0124, device='c

current epoch is:  6600
current loss is:  tensor(-0.7982, device='cuda:2')
current error is:  tensor(0.0008, device='cuda:2')
current epoch is:  6650
current loss is:  tensor(-0.7747, device='cuda:2')
current error is:  tensor(0.0013, device='cuda:2')
current epoch is:  6700
current loss is:  tensor(-0.7935, device='cuda:2')
current error is:  tensor(0.0020, device='cuda:2')
current epoch is:  6750
current loss is:  tensor(-0.7693, device='cuda:2')
current error is:  tensor(0.0030, device='cuda:2')
current epoch is:  6800
current loss is:  tensor(-0.7538, device='cuda:2')
current error is:  tensor(0.0010, device='cuda:2')
current epoch is:  6850
current loss is:  tensor(-0.7782, device='cuda:2')
current error is:  tensor(0.0004, device='cuda:2')
current epoch is:  6900
current loss is:  tensor(-0.8014, device='cuda:2')
current error is:  tensor(0.0013, device='cuda:2')
current epoch is:  6950
current loss is:  tensor(-0.7585, device='cuda:2')
current error is:  tensor(0.0013, device='c

current epoch is:  9900
current loss is:  tensor(-0.7624, device='cuda:2')
current error is:  tensor(0.0006, device='cuda:2')
current epoch is:  9950
current loss is:  tensor(-0.7515, device='cuda:2')
current error is:  tensor(0.0032, device='cuda:2')
total time is:  80.49766278266907 seconds


In [17]:
param = {}
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())
    print(parameters)

layer_out.weight : torch.Size([1, 1])
Parameter containing:
tensor([[-1.9974]], device='cuda:2', requires_grad=True)
layer_out.bias : torch.Size([1])
Parameter containing:
tensor([-0.9977], device='cuda:2', requires_grad=True)
